In [5]:
import numpy as np
import pandas as pd

 Dataset from Kaggle: https://www.kaggle.com/kazanova/sentiment140

In [12]:
csv=pd.read_csv("training.1600000.processed.noemoticon.csv",header=None,encoding='latin')

In [23]:
labels=list(csv.iloc[:,0])

In [73]:
for i in range(len(labels)):
    if labels[i]>0:
        labels[i]=1

In [74]:
def clean(s):
    x=filter(lambda x:x[0]!="@",s.split())
    x=filter(lambda x:x[:4]!="http",x)
    
    return " ".join(x)

In [75]:
tweets=list(csv.iloc[:,5].apply(clean))

In [76]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [77]:
vocab_size=20000
embedding_dim=100
max_length=12
train_size=1000000

In [78]:
trainx=tweets[:train_size]
trainy=labels[:train_size]
testx=tweets[train_size:]
testy=labels[train_size:]

In [79]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts(trainx)

In [80]:
sequence_train=tokenizer.texts_to_sequences(trainx)
sequence_test=tokenizer.texts_to_sequences(testx)
padded_train=pad_sequences(sequence_train,maxlen=max_length,truncating="post",padding="post")
padded_test=pad_sequences(sequence_test,maxlen=max_length,truncating="post",padding="post")

In [81]:
from keras import layers,models
model=models.Sequential()
model.add(layers.Embedding(vocab_size,embedding_dim,input_length=max_length))
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dense(28,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["acc"])

# Train on GPU if you dont have 6+ hours

In [ ]:
history=model.fit(padded_train,trainy,epochs=10,validation_data=(padded_test,testy))

In [83]:
import matplotlib.pyplot as plt
def loss_accuracy_graphs(history):
    acc=history.history['acc']
    val_accuracy=history.history['val_acc']
    loss==history.history['loss']
    val_loss=history.history['val_loss']
    plt.plot(acc,'b',label="training accuracy")
    plt.plot(val_accuracy,label="validation accuracy")    
    plt.legend()
    plt.figure()
    plt.plot(loss,'b',label="training loss")
    plt.plot(val_loss,label="validation loss")
    plt.legend()
    plt.show()

In [ ]:
loss_accuracy_graphs(history)